In [ ]:
import os
os.chdir('../')

In [ ]:
from utils import team_utils
import pickle
import pandas as pd
import numpy as np
from dspl.db import TeraDataDB

import os
import sys

import pandas as pd
import warnings
import gc

from dspl.models import classifiers
from scoring_pipeline.pipeline import JupyterNotebookPipeline
from scoring_pipeline.transformer import Estimator
warnings.simplefilter("ignore")
import os
import teradatasql
from joblib import Parallel, delayed
import multiprocessing
from functools import partial

In [ ]:
with open('score/for_load.pkl', 'rb') as handle:
    for_load = pickle.load(handle)

In [ ]:
# Гузим модели и энкедоры: ZIP(номер модели,кол-во фичей в ней)

models_path = dict()

for i, n in zip([0, 1, 2, 8], [8, 8, 6, 8]):
    models_path[i] = f'saveruns/less_features/runs/{i}/models/CatBoost.{n}.Perm_{i}.pkl'

for i, n in zip([3, 4, 5, 6, 7], [5, 18, 11, 11, 7]):
    models_path[i] = f'saveruns/less_features/runs/{i}/models/calib_CatBoost.{n}.Perm_{i}/isotonic.pkl'

encoders_path = dict()

for i, n in zip([0, 1, 2, 8], [8, 8, 6, 8]):
    encoders_path[i] = f'saveruns/less_features/runs/{i}/models/encoder.pkl'

for i, n in zip([3, 4, 5, 6, 7], [5, 18, 11, 11, 7]):
    encoders_path[i] = f'saveruns/less_features/runs/{i}/models/calib_CatBoost.{n}.Perm_{i}/encoder.pkl'

mod_enc = dict()
for i, j in zip(models_path, encoders_path):
    with open(models_path[i], "rb") as cds:
        model = pickle.load(cds)
    with open(encoders_path[j], "rb") as cds:
        encoder = pickle.load(cds)
    mod_enc[i] = [model, encoder]

In [ ]:
def score_proba(df, mod_enc):
    '''
    Функция для параллельного скоринга моделей


    Parameters
        ----------
        df: pandas.df
            Фичи для модели, не итерируются

        Returns
        -------
        mod_enc: model, lenght = 9
            На вход подается список моделей, по которым происходит параллельная итерация
    '''
    proc_id = multiprocessing.current_process()._identity[0]
    model = mod_enc[0]
    encoder = mod_enc[1]
    print('Read model_encoder', proc_id)
    df = encoder.transform(df)
    sc = model.transform(df)

    score = pd.DataFrame()
    score['epk_id'] = df.epk_id
    score[int(encoder.get_params()['config']['oot_data_path'][5])] = sc
    print('Score DONE', proc_id)
    return score

In [ ]:
if not os.path.exists('scoring_data/Data/batch'):
    os.makedirs('scoring_data/Data/batch')

In [ ]:
# Батчами по 700к наблюдейний грузим фичи, и параллельно скорим их 9ю моделями
for i, df in enumerate(pd.read_csv(for_load['path'],
                                   chunksize=700000,
                                   sep=';', header=None,
                                   names=df_crm.columns,
                                   dtype={'epk_id': 'int64'})):
    print(f'-------------------{i}------------------')
    if __name__ == '__main__':
        if i == 0:
            pool = multiprocessing.Pool(processes=9)
        df.to_csv(f'test.csv', index=False)

        f_score = partial(score_proba, df)
        score = pool.map(f_score, mod_enc.values())

        for j, df_score in enumerate(score):
            if j == 0:
                score_all_models = df_score
            else:
                score_all_models[df_score.columns[1]] = df_score.iloc[:, 1]
        del score
        score_all_models.to_csv(f'scoring_data/Data/batch/{i}.csv', index=False)
        del df  
        del score_all_models

pool.close()
pool.join()
del pool 